In [1]:
## Tested using the following loaded modules
## module use /g/data/access/ngm/modules
## module load analysis3/21.10
import os, sys
workdir = "/g/data/xv83/users/bxn599/ACS/evaluation/"
os.chdir(workdir)
sys.path.append("./lib")
import glob
import xarray as xr
import numpy as np
import geopandas as gp
from datetime import datetime as dt
import lib_standards
import lib_spatial
import spatial_selection
import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
importlib.reload(lib_standards)
import warnings
warnings.filterwarnings('ignore')
import csv
import ccam_drs_interface
import re

font = {'size'   : 12}
mpl.rc('font', **font)

In [2]:
DATA_LOCATIONS_HIST = {
    'BARPA-R:ACCESS-CM2': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/CSIRO-ARCCSS-ACCESS-CM2/historical/r4i1p1f1/BOM-BARPA-R/v1/climdex",
    'BARPA-R:ACCESS-ESM1-5': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/CSIRO-ACCESS-ESM1-5/historical/r6i1p1f1/BOM-BARPA-R/v1/climdex",
    'BARPA-R:CESM2': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/NCAR-CESM2/historical/r11i1p1f1/BOM-BARPA-R/v1/climdex",
    'BARPA-R:CMCC-ESM2': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/CMCC-CMCC-ESM2/historical/r1i1p1f1/BOM-BARPA-R/v1/climdex",
    'BARPA-R:CNRM-ESM2-1': "", # not downscaled by BoM
    'BARPA-R:EC-Earth3': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/EC-Earth-Consortium-EC-Earth3/historical/r1i1p1f1/BOM-BARPA-R/v1/climdex",
    'BARPA-R:MPI-ESM1-2-HR': "", # post processing in progress
    'BARPA-R:NorESM2-MM': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/NCC-NorESM2-MM/historical/r1i1p1f1/BOM-BARPA-R/v1/climdex",
}
DATA_LOCATIONS_FUT = {
    'BARPA-R:ACCESS-CM2': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/CSIRO-ARCCSS-ACCESS-CM2/ssp370/r4i1p1f1/BOM-BARPA-R/v1/climdex",
    'BARPA-R:ACCESS-ESM1-5': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/CSIRO-ACCESS-ESM1-5/ssp370/r6i1p1f1/BOM-BARPA-R/v1/climdex",
    'BARPA-R:CESM2': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/NCAR-CESM2/ssp370/r11i1p1f1/BOM-BARPA-R/v1/climdex",
    'BARPA-R:CMCC-ESM2': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/CMCC-CMCC-ESM2/ssp370/r1i1p1f1/BOM-BARPA-R/v1/climdex",
    'BARPA-R:CNRM-ESM2-1': "", # not downscaled by BoM
    'BARPA-R:EC-Earth3': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/EC-Earth-Consortium-EC-Earth3/ssp370/r1i1p1f1/BOM-BARPA-R/v1/climdex",
    'BARPA-R:MPI-ESM1-2-HR': "", # post processing in progress
    'BARPA-R:NorESM2-MM': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-15/BOM/NCC-NorESM2-MM/ssp370/r1i1p1f1/BOM-BARPA-R/v1/climdex",
}
# Frequency of icclim indicator
FREQ = 'month'
# Data sources to consider here
SOURCES_HIST = ['BARPA-R:EC-Earth3', 'BARPA-R:NorESM2-MM']

SOURCES_FUT = SOURCES_HIST

# Grid to used as the reference grid to compute difference metrics
REFERENCE_GRID = 'BARPA-R:NorESM2-MM'
# "Truth" for compute difference metrics
REFERENCE = 'AGCD'
# Region of interest, as per defined in lib_standards.DOMAINS
REGION = 'Australia'  
# Period of interest, as per defined in lib_standards.PERIODS
EARLYPERIOD = 'ACS_HISTORICAL' 
LATEPERIOD = 'ACS_2020'
SEASON='all'

In [3]:
def prepare_data_hist(index, sources=SOURCES_HIST, freq=FREQ, region=REGION, period=EARLYPERIOD, reference_grid=REFERENCE_GRID, season=SEASON):
    """
    Reads in the icclim data and regrid to the reference grid.
    """
    # Load the data sources
    ds = {}
    for s in sources:
        fs = glob.glob(os.path.join(DATA_LOCATIONS_HIST[s], index, "{index}_*_{freq}_*.nc".format(index=index, freq=freq)))
        ds[s] = xr.open_dataset(fs[0])
        ds[s] = lib_standards.standardise_data(ds[s], region=region, period=period, season=season, compute=True)
        experiment=re.findall('(?<=historical/)(.*)(?=/BOM-BARPA-R)',DATA_LOCATIONS_HIST[s])
    return ds, experiment

In [4]:
def prepare_data_fut(index, sources=SOURCES_FUT, freq=FREQ, region=REGION, period=LATEPERIOD, reference_grid=REFERENCE_GRID, season=SEASON):
    """
    Reads in the icclim data and regrid to the reference grid.
    """
    # Load the data sources
    ds = {}
    for s in sources:
        fs = glob.glob(os.path.join(DATA_LOCATIONS_FUT[s], index, "{index}_*_{freq}_*.nc".format(index=index, freq=freq)))
        ds[s] = xr.open_dataset(fs[0])
        ds[s] = lib_standards.standardise_data(ds[s], region=region, period=period, season=season, compute=True)
    return ds

In [5]:
header = ['Season', 'Variable', 'Project', 'Domain', 'Downscaler', 'Region', 'Model', 'Experiment', 'Variant', 'Hist-Epoch', 'Proj-Epoch', 'Change-Type', 'Value']
f = open('/g/data/xv83/users/bxn599/ACS/evaluation/test_barpa.csv','w')
writer = csv.writer(f)
writer.writerow(header)
INDEXES = ['PRCPTOT', 'PRCPTOT:DJF', 'PRCPTOT:JJA', 'PRCPTOT:MAM', 'PRCPTOT:SON']
LATEPERIOD = {'ACS_2020': '2020-2039', 'ACS_2040': '2040-2069', 'ACS_2060': '2060-2079', 'ACS_2080': '2080-2099'}
for PERIOD in LATEPERIOD:
    for index in INDEXES:
        futperiod=LATEPERIOD[PERIOD]
        print("index = {:}".format(index))
        # check whether the index has special conditions
        index_condition = None
        if not ":" in index:
            # Prepare all the icclim data as per the setup in CONFIGURATIONS
            ds0, experiment = prepare_data_hist(index)
            ds1 = prepare_data_fut(index, period=PERIOD)
            clabel = "% change of {index} ({freq})".format(index=index, freq=FREQ)
            season = 'annual'
        else:
            index_condition = index.split(":")[1]
            index = index.split(":")[0]
            ds0, experiment = prepare_data_hist(index, season=index_condition)
            ds1 = prepare_data_fut(index, season=index_condition, period=PERIOD)
            clabel = "% change of {index} ({index_condition})".format(index=index, index_condition=index_condition)
            season = index_condition

        # print(futperiod)
        # print(model)
        # print(experiment[0])
        # print(season)
        ds0_target = {}
        ds1_target = {}
        ds_target = {}
        flist = []
        for s0, s1 in zip(SOURCES_HIST, SOURCES_FUT):
            print(s0, s1)
            model=s0.split(":")[1]
            # print(model)
            # print("test")
            ds0_target[s0] = ds0[s0][index].mean(dim='time')
            ds0_target[s0] = lib_spatial.apply_region_mask(ds0_target[s0], 'Australia')
            ds1_target[s1] = ds1[s1][index].mean(dim='time')
            ds1_target[s1] = lib_spatial.apply_region_mask(ds1_target[s1], 'Australia')

            ds_target[s0] = (ds1_target[s1]-ds0_target[s0])/ds0_target[s0]*100 # percentage change
            #print(ds_target[s0])
            weights = np.cos(np.deg2rad(ds_target[s0].lat))
            weights.name = "weights"
            area_avg = ds_target[s0].weighted(weights)
            area_avg = ds_target[s0].mean(("lon", "lat"))
            print(experiment)
            print(futperiod)
            print(area_avg.values)
            data = [season, 'pr', 'BARPA-R', 'AUS-15', 'n/a', 'Australia', model, 'ssp370', experiment[0], '1995-2014', futperiod, 'percent',area_avg.values]
            writer.writerow(data)

index = PRCPTOT
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
['r1i1p1f1']
2020-2039
22.94797585451271
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
['r1i1p1f1']
2020-2039
3.2600064968312745
index = PRCPTOT:DJF
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
['r1i1p1f1']
2020-2039
26.860760667391514
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
['r1i1p1f1']
2020-2039
6.7924327572728185
index = PRCPTOT:JJA
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
['r1i1p1f1']
2020-2039
17.391169899882218
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
['r1i1p1f1']
2020-2039
-7.03415987688309
index = PRCPTOT:MAM
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
['r1i1p1f1']
2020-2039
26.801575954181104
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
['r1i1p1f1']
2020-2039
21.947410450058825
index = PRCPTOT:SON
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
['r1i1p1f1']
2020-2039
18.341513046040028
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
['r1i1p1f1']
2020-2039
-7.749124861599771
index = PRCPTOT
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
['r1i1p1f1']
2040-2069
25.286489201864608
BARPA-R:NorESM2-MM BAR

In [6]:
INDEXES = ['TG', 'TG:DJF', 'TG:JJA', 'TG:MAM', 'TG:SON']
LATEPERIOD = {'ACS_2020': '2020-2039', 'ACS_2040': '2040-2069', 'ACS_2060': '2060-2079', 'ACS_2080': '2080-2099'}
for PERIOD in LATEPERIOD:
    for index in INDEXES:
        futperiod=LATEPERIOD[PERIOD]
        print("index = {:}".format(index))
        # check whether the index has special conditions
        index_condition = None
        if not ":" in index:
            # Prepare all the icclim data as per the setup in CONFIGURATIONS
            ds0, experiment = prepare_data_hist(index)
            ds1 = prepare_data_fut(index, period=PERIOD)
            clabel = "% change of {index} ({freq})".format(index=index, freq=FREQ)
            season = 'annual'
        else:
            index_condition = index.split(":")[1]
            index = index.split(":")[0]
            ds0, experiment = prepare_data_hist(index, season=index_condition)
            ds1 = prepare_data_fut(index, season=index_condition, period=PERIOD)
            clabel = "% change of {index} ({index_condition})".format(index=index, index_condition=index_condition)
            season = index_condition

        # print(futperiod)
        # print(model)
        # print(experiment[0])
        # print(season)
        ds0_target = {}
        ds1_target = {}
        ds_target = {}
        flist = []
        for s0, s1 in zip(SOURCES_HIST, SOURCES_FUT):
            print(s0, s1)
            model=s0.split(":")[1]
            # print(model)
            # print("test")
            ds0_target[s0] = ds0[s0][index].mean(dim='time')
            ds0_target[s0] = lib_spatial.apply_region_mask(ds0_target[s0], 'Australia')
            ds1_target[s1] = ds1[s1][index].mean(dim='time')
            ds1_target[s1] = lib_spatial.apply_region_mask(ds1_target[s1], 'Australia')

            ds_target[s0] = (ds1_target[s1]-ds0_target[s0]) # absolute change
            #print(ds_target[s0])
            weights = np.cos(np.deg2rad(ds_target[s0].lat))
            weights.name = "weights"
            area_avg = ds_target[s0].weighted(weights)
            area_avg = ds_target[s0].mean(("lon", "lat"))
            print(area_avg.values)
            data = [season, 'tg', 'BARPA-R', 'AUS-15', 'n/a', 'Australia', model, 'ssp370', experiment[0], '1995-2014', futperiod, 'absolute', area_avg.values]
            writer.writerow(data)

index = TG
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
0.8166626
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
0.29525998
index = TG:DJF
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
0.71146876
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
0.41709948
index = TG:JJA
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
1.0053643
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
0.2690085
index = TG:MAM
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
0.84705055
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
0.012424965
index = TG:SON
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
0.70276684
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
0.4825074
index = TG
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
1.4585028
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
1.2303035
index = TG:DJF
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
1.4913101
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
1.3052845
index = TG:JJA
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
1.3846778
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
0.7973625
index = TG:MAM
BARPA-R:EC-Earth3 BARPA-R:EC-Earth3
1.5913316
BARPA-R:NorESM2-MM BARPA-R:NorESM2-MM
1.2026047
index = TG:SON
BARPA

In [7]:
f.close()